In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from datetime import datetime
from os import listdir
from os.path import isfile, join

import librosa
import librosa.display
import IPython.display as ipd

import cv2

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Flatten, LSTM, TimeDistributed
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
mypath = "/content/drive/MyDrive/Respiratory_Sound_Database/audio_and_txt_files/"
filenames = [file for file in listdir(mypath) if (isfile(join(mypath, file)) and file.endswith('.wav'))] 
#filenames

In [ ]:
p_id_in_file = [] # patient IDs corresponding to each file
for name in filenames:
    p_id_in_file.append(int(name[:3]))

p_id_in_file = np.array(p_id_in_file) 
#p_id_in_file

In [ ]:
# Adding white noise 
def add_noise(data):
    wn = np.random.randn(len(data))
    data_wn = data + 0.002*wn
    return data_wn

# Shifting the sound
def shift_sound(data):
    data_shift = np.roll(data, 1600)
    return data_shift

#streching
def stretch(data, rate=0.98):
    data = librosa.effects.time_stretch(data, rate)
    return data

#adding pitch
def pitch(data,sr):
    return librosa.effects.pitch_shift(data, sr, n_steps=2)

In [ ]:
max_pad_len = 862 # to make the length of all MFCC equal

def extract_features(file_name,label):
    # file_name='/content/drive/MyDrive/Respiratory_Sound_Database/audio_and_txt_files/166_1p1_Ar_sc_Meditron.wav'
    try:
        mfccs_all = []
        allAudio=None
        audio, sample_rate = librosa.load(file_name) 
        if not label == 'COPD':
            noise_audio = add_noise(audio)
            shift_audio = shift_sound(audio)
            stretch_audio = stretch(audio)
            pitch_audio = pitch(audio,sample_rate)
            allAudio = [audio,noise_audio,shift_audio,stretch_audio,pitch_audio]

        else:
            allAudio = [audio]
        
        for single_audio in allAudio:
            mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
            pad_width = max_pad_len - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
            mfccs_all.append(mfccs)
             
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs_all

In [ ]:
filepaths = [join(mypath, f) for f in filenames] # full paths of files'
#filepaths

In [ ]:
p_diag = pd.read_csv("/content/drive/MyDrive/Respiratory_Sound_Database/patient_diagnosis.csv",header=None)

In [ ]:
#print(p_diag[p_diag[0]==160][1].values[0])
labels = np.array([p_diag[p_diag[0] == x][1].values[0] for x in p_id_in_file])
# print(labels)

In [ ]:
features = [] 
newLabels = []
i = 0
# Iterate through each sound file and extract the features
for file_name in filepaths:
    data = extract_features(file_name,labels[i])
    print(labels[i])#,data)
    features.extend(data)
    if labels[i] == 'COPD':
        newLabels.append('COPD')
    else:
        label = labels[i]
        newLabels.extend([label,label,label,label,label])
    i+=1
print('Finished feature extraction from ', len(features), ' files')
print('Finished feature extraction from ', len(newLabels), ' labels')
features = np.array(features)
labels = np.array(newLabels)

In [ ]:
plt.figure(figsize=(10, 4))
librosa.display.specshow(features[7], x_axis='time')
plt.colorbar()
plt.title('MFCC')
plt.tight_layout()
plt.show()